In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Import data
file_path = 'C:/Users/User/Documents/SKRIPSI/OPTIMASI PER PENYAKIT/Dataset_Skripsi_Processed.csv'  # Path absolut
df = pd.read_csv(file_path)

In [3]:
# Set seed for reproducibility
np.random.seed(42)

## Inisialisasi Parameter

In [4]:
w_max = 0.7  # Bobot inersia maksimal
w_min = 0.3  # Bobot inersia minimal
c1 = 1.5  # Koefisien akselerasi 1
c2 = 1.5  # Koefisien akselerasi 2
n_particles = 100  # Jumlah partikel
n_iterations = 50  # Jumlah iterasi

In [5]:
# Filter data untuk P06 saja
df_P06 = df[df['kode_penyakit'] == 'P06']  # Ambil data hanya untuk P06
cf_pakar = df_P06['cf_pakar'].values  # Mengambil data cf_pakar untuk P06

## Fungsi Fitness

In [6]:
# Fungsi fitness untuk mengukur sejauh mana posisi partikel mendekati cf_pakar
def fitness(position, cf_pakar):
    return np.mean((cf_pakar - position) ** 2)  # MSE 

## Implementasi PSO

In [7]:
def run_pso_get_gbest(cf_pakar):
    # Inisialisasi kecepatan dan posisi partikel secara acak
    position = np.random.uniform(0.0, 1.0, (n_particles, len(cf_pakar)))  # Posisi partikel
    velocity = np.random.uniform(-0.1, 0.1, (n_particles, len(cf_pakar)))  # Kecepatan partikel

    # Inisialisasi pbest dan gbest
    pbest = position.copy()  # Set pbest awal sama dengan posisi partikel
    pbest_score = np.array([fitness(p, cf_pakar) for p in position])  # Nilai fitness awal untuk pbest
    gbest = pbest[np.argmin(pbest_score)]  # gbest adalah posisi dengan nilai fitness terbaik
    gbest_score = np.min(pbest_score)  # Nilai fitness terbaik untuk gbest

    # Iterasi PSO
    for iteration in range(n_iterations):
        w = w_max - (w_max - w_min) * (iteration / n_iterations)  # Interpolasi w antara w_max dan w_min

        # Update posisi dan kecepatan setiap partikel
        for i in range(n_particles):
            r1 = np.random.rand(len(cf_pakar))  # Random number untuk C1
            r2 = np.random.rand(len(cf_pakar))  # Random number untuk C2

            # Update kecepatan
            velocity[i] = w * velocity[i] + c1 * r1 * (pbest[i] - position[i]) + c2 * r2 * (gbest - position[i])

            # Update posisi
            position[i] += velocity[i]

            # Pastikan posisi berada dalam rentang 0 dan 1
            position[i] = np.clip(position[i], 0.0, 1.0)

        # Evaluasi fitness setiap partikel
        score = np.array([fitness(p, cf_pakar) for p in position])

        # Update pbest dan gbest
        for i in range(n_particles):
            if score[i] < pbest_score[i]:
                pbest[i] = position[i]  # Update pbest dengan posisi partikel yang lebih baik
                pbest_score[i] = score[i]

        # Update gbest jika ditemukan pbest yang lebih baik
        gbest_score_new = np.min(pbest_score)
        if gbest_score_new < gbest_score:
            gbest = pbest[np.argmin(pbest_score)]  # Update gbest dengan posisi pbest terbaik
            gbest_score = gbest_score_new

        # Tampilkan kecepatan dan posisi setelah update pada iterasi ke-60
        if iteration == n_iterations - 1:
            display_final_result(gbest, gbest_score, position, velocity, iteration, cf_pakar)

    return gbest, gbest_score

# Fungsi untuk menampilkan hasil akhir pada iterasi terakhir
def display_final_result(gbest, gbest_score, position, velocity, iteration, cf_pakar):
    """Menampilkan hasil akhir pada iterasi ke-60"""
    print(f"\nFitness (gbest iterasi akhir): {gbest_score}")
    print("Nilai CF untuk setiap gejala (G01 - G31):")
    
    result_data = {
        'Penyakit': 'P06',  # Ganti dengan nama penyakit sesuai
        'Fitness (gbest iterasi akhir)': round(gbest_score, 5),
    }
    
    # Menambahkan nilai CF untuk setiap gejala
    for i, g in enumerate(gbest):
        result_data[f'G{i+1}'] = g  # Menampilkan nilai CF per gejala tanpa pembatasan angka di belakang koma
    
    # Menampilkan hasil akhir (tidak menampilkan velocity dan posisi per iterasi)
    print(f"\nNilai CF setelah update (Iterasi ke-{n_iterations}):")
    print(result_data)
    
    # Menyimpan hasil ke CSV
    output_file = 'hasil_optimasi_pso_P06.csv'
    df_result = pd.DataFrame([result_data])  # Menyusun hasil dalam DataFrame
    if os.path.exists(output_file):
        df_result.to_csv(output_file, mode='a', header=False, index=False)  # Menambahkan ke file yang sudah ada
    else:
        df_result.to_csv(output_file, mode='w', header=True, index=False)  # Menyimpan sebagai file baru jika belum ada


In [8]:
# Jalankan PSO untuk P06
gbest, gbest_score = run_pso_get_gbest(cf_pakar)


Fitness (gbest iterasi akhir): 0.0038709677419358963
Nilai CF untuk setiap gejala (G01 - G31):

Nilai CF setelah update (Iterasi ke-50):
{'Penyakit': 'P06', 'Fitness (gbest iterasi akhir)': 0.00387, 'G1': 0.7999999079095587, 'G2': 0.0, 'G3': 0.8000000659652396, 'G4': 1.0, 'G5': 1.0, 'G6': 0.0, 'G7': 0.0, 'G8': 0.0, 'G9': 0.0, 'G10': 0.0, 'G11': 0.0, 'G12': 0.0, 'G13': 0.0, 'G14': 0.0, 'G15': 1.0, 'G16': 0.0, 'G17': 1.0, 'G18': 1.0, 'G19': 1.0, 'G20': 0.0, 'G21': 0.0, 'G22': 0.0, 'G23': 0.0, 'G24': 0.0, 'G25': 0.0, 'G26': 0.0, 'G27': 0.0, 'G28': 0.0, 'G29': 0.0, 'G30': 0.0, 'G31': 0.0}
